In [1]:
import torch
torch.cuda.is_available()


True

# model

In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Train

In [3]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


# Test

In [4]:

def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



# main

In [5]:
torch.manual_seed(1)
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, 
#     transform=transforms.Compose([transforms.ToTensor(),
#                                   transforms.Normalize((0.1307,), (0.3081,))])),
    transform=transforms.Compose([transforms.ToTensor()])),

    batch_size=64,  
    shuffle=True, 
    num_workers = 1, 
    pin_memory = True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False,
    transform=transforms.Compose([transforms.ToTensor()])),
#     transform=transforms.Compose([transforms.ToTensor(),
#                                   transforms.Normalize((0.1307,), (0.3081,))])),
    batch_size=1000, 
    shuffle=True, 
    num_workers = 1, 
    pin_memory = True)

model = Net().cuda()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

for epoch in range(1, 50 + 1):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.324429
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.310352
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.301021
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.331953
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.298500
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.292652
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.297015
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.308769
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.289417
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.279603
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.307612
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.289493
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.273442
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.284090
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.297391
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.265354
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.271982
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.262379
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.270753
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.236508
Train Epoch: 1 [12800/60000 (

/srv/apps/anaconda3/envs/py35/lib/python3.5/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.3490, Accuracy: 9015/10000 (90%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.572061
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.423386
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.513301
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.793961
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.620902
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.523630
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.556751
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.641969
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.483842
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.793341
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.593476
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.586509
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.768875
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.413104
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.540128
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.551968
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.732913
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.455457
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.501786
Train Epoch: 2 [12